In [1]:
!pip install spotipy

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
!pip install fa2

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pprint
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import time

In [4]:
import networkx as nx
from fa2 import ForceAtlas2

In [5]:
!pip install azlyrics

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
!pip install lyricwikia

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [7]:
!pip install BeautifulSoup4

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
from azlyrics import lyrics
import pandas as pd
import re
import lyricwikia
import urllib2
import json
import bs4
import requests
import unicodedata
import nltk

In [9]:
mkdir lyrics

A subdirectory or file lyrics already exists.


In [10]:
mkdir topsters

A subdirectory or file topsters already exists.


In [11]:
#SETTING THE DATAFRAMES WE NEED HERE
artists_df = pd.read_csv('artists.csv')

In [12]:
client_id = "b06999e849764d0cb9d85ff2b4762fd9"
client_secret = "4565b9044d694deb9cf54eddb9b08e69"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [13]:
len(artists_df.index)

664

In [ ]:
artist_dict = {}
counter = 0
for index, row in artists_df.iterrows():
    artist_id = row['Artist_id']
    artist_name = row['Artist']
    if (artist_id in artist_dict):
        print(artist_id)
        print(artist_name)
        print(artist_dict[artist_id])
        counter += 1
        continue
    artist_dict[artist_id] = artist_name
print(counter)

In [14]:
counter = 0
counter_dup = 0
import os.path
for index, row in artists_df.iterrows():
    counter += 1
    if (counter % 10 == 0):
        print(counter)
    artist_id = row['Artist_id']
    artist_name = row['Artist']
    try:
        if (os.path.exists("./topsters/songs_"+str(artist_id)+".csv")):
            print(artist_id)
            counter_dup += 1
        fileCsv2 = csv.writer(open("./topsters/songs_"+str(artist_id)+".csv", "w+"))
        fileCsv2.writerow(['Artist', 'Artist_id', 'Song', 'Song_id'])
        top_track_list = sp.artist_top_tracks(str(artist_id))['tracks']
    except:
        print(artist_id)
        pass
    for track in top_track_list:
        row = [artist_name.encode('ascii', 'ignore'),str(artist_id).encode('ascii', 'ignore'),track['name'].encode('ascii', 'ignore'), track['id'].encode('ascii', 'ignore')]
        fileCsv2.writerow(row)
print(counter_dup)

0Y5tJX1MQlPlqiwlOH1tJY
3TVXtAsR1Inumwj472S9r4
66CXWjxzNUsdJxJ2JdwvnR
0hCNtLu0JehylgoiP8L4Gh
246dkjvS1zLTtiykXe5h60
6LuN9FCkKOj5PcnpouEgny
55Aa2cqylxrFIXC767Z865
5f7VJjfbwm532GiveGC0ZK
4kYSro6naA4h99UJvo89HB
10
1RyvyyTE3xzB2ZywiAwp0i
46SHBwWsqBkxI7EeeBEQG7
0VRj0yCOv2FXJNP47XQnx5
6qqNVTkY8uBg9cP3Jd7DAH
1Xyo4u8uXC1ZmMpatF05PJ
5K4W6rqBFWDnAN6FQUkS6x
4O15NlyKLIASxsJ0PrXPfz
6eUKZXaKkcviH0Ku9w2n3V
1HY2Jd0NmPuamShAr6KMms
26VFTg2z8YR0cCuwLzESi2
20
0iEtIxbK0KxaSlF7G42ZOp
06HL4z0CvFAxyc27GXpf02
2hlmm7s2ICUX0LVIhVFlZQ
0du5cEVh5yTK9QJze8zA0C
7bXgB6jMjp9ATFy66eO08Z
3Nrfpe0tUJi4K4DXYWgMUX
1URnnhqYAYcrqrcwql10ft
50co4Is1HCEo8bhOyUWKpn
7c0XG5cIJTrrAgEC3ULPiq
5pKCCKE2ajJHZ9KAiaK11H
30
1uNFoZAHBGtllmzznpCI3s
3YQKmKGau1PzlVlkL1iodx
7n2wHs1TKAczGzO7Dd2rGr
04gDigrS5kc9YWfZHwBETP
6oMuImdp5ZcFhWP0ESe6mG
02kJSzxNuaWGqwubyUba0Z
4LLpKhyESsyAXpc4laK94U
0C8ZW7ezQVs4URX5aX7Kqx
4nDoRrQiYLoBzwC5BhVJzF
2wY79sveU1sp5g7SokKOiI
40
20JZFwl6HVl6yg8a4H3ZqK
64KEffDW9EtZ1y2vBYgq8T
6M2wZ9GZgrQXHCFfjv46we
13y7CgLHjMVRMDqxdx0Xdo

In [15]:
genius_api_dict = {'id':'eOFdp-v__I7JeKporMF3K49wKnz3cYIkJcOGUP1wRz2uV7AmWh1eeBup_zJMCkqA',
              'secret': 'kbbVWQxhXtsfnHm1W65gkjnvxlt9A3U2ih5bf6KWBJe6_KeYWZWyxC4rDK62OX9OQ9cLWTH0FgKZWoRmEofH9g',
              'access_token': 'jhpqKVzO880gvQ5i-JkEh08wTUhmmA4kMuCkt5MQKLKMrWnjjqs3Z0WiVd49TmR7'}

In [16]:
def genius_Lyrics(artist, song):
    querystring = "http://api.genius.com/search?q=" + urllib2.quote(song + " " + artist)
    request = urllib2.Request(querystring)
    request.add_header("Authorization", "Bearer " + genius_api_dict['access_token'])   
    request.add_header("User-Agent", "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)") #Must include user agent of some sort, otherwise 403 returned
    while True:
        try:
            response = urllib2.urlopen(request, timeout=4) #timeout set to 4 seconds; automatically retries if times out
            raw = response.read()
        except socket.timeout:
            print("Timeout raised and caught")
            continue
        break
    json_obj = json.loads(raw)
    body = json_obj["response"]["hits"]

    num_hits = len(body)
    if num_hits==0:
        print("\t\tNo results for: " + song)      

    for result in body:

#             result_id = result["result"]["id"]
#             title = result["result"]["title"]
        url = result["result"]["url"]
#             path = result["result"]["path"]
#             header_image_url = result["result"]["header_image_url"]
#             annotation_count = result["result"]["annotation_count"]
#             pyongs_count = result["result"]["pyongs_count"]
#             primaryartist_id = result["result"]["primary_artist"]["id"]
#             primaryartist_name = result["result"]["primary_artist"]["name"]
#             primaryartist_url = result["result"]["primary_artist"]["url"]
#             primaryartist_imageurl = result["result"]["primary_artist"]["image_url"]
#             row=[page,result_id,title,url,path,header_image_url,annotation_count,pyongs_count,primaryartist_id,primaryartist_name,primaryartist_url,primaryartist_imageurl]
#             print(row) #write as CSV
        page = requests.get(url)
        if page.status_code == 404:
            return None

        # Scrape the song lyrics from the HTML
        html = bs4.BeautifulSoup(page.text, "html.parser")
        title_genius = html.find("h1", class_="header_with_cover_art-primary_info-title").get_text().strip()
        artist_genius = html.find("a", class_="header_with_cover_art-primary_info-primary_artist").get_text().strip()
#         print('artist_genius: {} - artist: {}'.format(artist_genius, artist))
        if (title_genius == song.strip() ):#and artist_genius == artist):
            lyrics = html.find("div", class_="lyrics").get_text()
            lyrics = re.sub('(\[.*?\])*', '', lyrics)
            lyrics = re.sub('\n{2}', '\n', lyrics)  # Gaps between verses

            lyrics = unicodedata.normalize('NFKD', lyrics).encode('ascii','ignore')
            return lyrics.strip('\n')
    return None    
        

In [17]:
def removePunctuation(tokens):
    """
    This function finds punctuation (apart from '#' symbols) from each string token inside the provided token list
    
    arguments: tokens: list of strings
    returns: list of strings (tokens)
    """
    new_token_list = []
    for token in tokens:
        #returns a new string without punctuation
        new_token = re.sub(r'[^\w\s#]','',token)
        if (len(new_token) > 0):
            new_token_list.append(new_token)
    return new_token_list

In [18]:
def getLyricsAllSongs(artists_df):
    for index, row in artists_df.iterrows():
        artist_id = row['Artist_id']
        artist_name = row['Artist']
        artists_song_df = pd.read_csv('./songs/songs_'+artist_id+'.csv')
        getLyrics(artist_id, artist_name, artists_song_df)

        
def getLyricsTopsters(artists_df):
    for index, row in artists_df.iterrows():
        artist_id = row['Artist_id']
        artist_name = row['Artist']
        artists_song_df = pd.read_csv('./topsters/songs_'+artist_id+'.csv')
        getLyrics(artist_id, artist_name, artists_song_df)


In [19]:
def getLyrics(artist_id, artist_name, artists_song_df):
    fileCsv_song_lyrics = csv.writer(open('./lyrics/lyrics_'+artist_id+'.csv', "w+"))
    fileCsv_song_lyrics.writerow(['Artist', 'Artist_id','Song','Song_id','Lyrics'])
    unique_song_list = []
    for index_song, row_song in artists_song_df.iterrows():
    #         print(row_song['Song'])
        if (not(row_song['Song'] in unique_song_list)):
            unique_song_list.append(row_song['Song'])
            lyrics = None
            if (not('strumental' in row_song['Song'] and 'emix' in row_song['Song'])):
                song_name_wo_parentheses = re.sub(r'\([^()]*\)', '', row_song['Song'])
                print artist_name
                print "===="+song_name_wo_parentheses
                try:
                    wd = lyrics(artist_name, song_name_wo_parentheses)
                    if ('Error' in wd):
                        wd = lyrics(artist_name, row_song['Song'])
                    if ('Error' in wd):
                        raise Exception
            #             print(song_name_wo_parentheses +' - Unable to find lyrics in AZ')
                    else:
                        lyrics = wd
                        print(row_song['Song'] +' - FOUND LYRICS in AZ')

                except: #if shadow banned from AZLyrics
                    print('\tAZ failed')
                    try:
                        lyrics = lyricwikia.get_lyrics(artist_name, song_name_wo_parentheses)
                    except:
                        print('\tLyric Wikia failed')
                        try:
                            lyrics = genius_Lyrics(artist_name, song_name_wo_parentheses)
                            if (lyrics == None):
                                print("\tGenius failed")
                        except:
                            print('Socket error - Sleeping 5 sec')
                            time.sleep(5)
            if not(lyrics == None):
                #tokenize lyrics
                lyrics = nltk.word_tokenize(lyrics)
                lyrics = removePunctuation(lyrics)
                if isinstance(lyrics, list):
                    lyric_string = ""
                    for lyric in lyrics:
                        lyric_string = lyric_string + " " + lyric
                    lyrics = lyric_string
                fileCsv_song_lyrics.writerow([artist_name, artist_id,row_song['Song'],row_song['Song_id'],lyrics])           #do NOT write in the CSV if you cannot find the lyrics
                print("\tADDED LYRICS")

In [20]:
getLyricsTopsters(artists_df)

Travis Scott
====SICKO MODE
	AZ failed
	ADDED LYRICS
Travis Scott
====STARGAZING
	AZ failed
	ADDED LYRICS
Travis Scott
====YOSEMITE
	AZ failed
	ADDED LYRICS
Travis Scott
====goosebumps
	AZ failed
	ADDED LYRICS
Travis Scott
====BUTTERFLY EFFECT
	AZ failed
	ADDED LYRICS
Travis Scott
====CAN'T SAY
	AZ failed
	ADDED LYRICS
Travis Scott
====WAKE UP
	AZ failed
	ADDED LYRICS
Travis Scott
====NO BYSTANDERS
	AZ failed
	ADDED LYRICS
Travis Scott
====STOP TRYING TO BE GOD
	AZ failed
	ADDED LYRICS
Travis Scott
====R.I.P. SCREW
	AZ failed
	ADDED LYRICS
Drake
====In My Feelings
	AZ failed
	ADDED LYRICS
Drake
====Nonstop
	AZ failed
	ADDED LYRICS
Drake
====God's Plan
	AZ failed
	ADDED LYRICS
Drake
====Never Recover 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Drake
====Yes Indeed
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Drake
====Nice For What
	AZ failed
	ADDED LYRICS
Drake
====Look Alive 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Drake
====Dont Matter To Me 
	AZ failed
	ADDED LYRICS
Drake
===

Ty Dolla $ign
====The Light
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Ty Dolla $ign
====Pineapple 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Ty Dolla $ign
====Love U Better 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Ty Dolla $ign
====Bad Bitch 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Ty Dolla $ign
====Shootin Shots 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Ty Dolla $ign
====Wavy 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Rihanna
====Umbrella
	AZ failed
	ADDED LYRICS
Rihanna
====FourFiveSeconds
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Rihanna
====Take A Bow
	AZ failed
	ADDED LYRICS
Rihanna
====Love On The Brain
	AZ failed
	ADDED LYRICS
Rihanna
====Don't Stop The Music
	AZ failed
	ADDED LYRICS
Rihanna
====Bitch Better Have My Money
	AZ failed
	ADDED LYRICS
Rihanna
====Pon de Replay
	AZ failed
	ADDED LYRICS
Rihanna
====Work
	AZ failed
	ADDED LYRICS
Rihanna
====Stay - Album Version 
	AZ failed
	Lyric Wikia failed
	Genius failed
Rihanna
====Needed Me
	AZ failed
	ADD

====Crazy In Love
	AZ failed
	Lyric Wikia failed
	Genius failed
Beyonc
====Irreplaceable
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Beyonc
====Love On Top
	AZ failed
	Lyric Wikia failed
	Genius failed
Beyonc
====Run the World 
	AZ failed
	Lyric Wikia failed
	Genius failed
Beyonc
====7/11
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Beyonc
====Drunk in Love
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Beyonc
====Best Thing I Never Had
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Beyonc
====Partition
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Demi Lovato
====Solo 
	AZ failed
	ADDED LYRICS
Demi Lovato
====Sorry Not Sorry
	AZ failed
	ADDED LYRICS
Demi Lovato
====chame La Culpa
	AZ failed
	Lyric Wikia failed
		No results for: chame La Culpa
	Genius failed
Demi Lovato
====Sober
	AZ failed
	ADDED LYRICS
Demi Lovato
====Stone Cold
	AZ failed
	ADDED LYRICS
Demi Lovato
====Tell Me You Love Me
	AZ failed
	ADDED LYRICS
Demi Lovato
====Confident
	AZ failed
	ADDED LYRICS
Demi Lovato
====Heart At

PnB Rock
====Issues 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
PnB Rock
====ABCD 
	AZ failed
	Lyric Wikia failed
	Genius failed
PnB Rock
====Nowadays
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
PnB Rock
====Misunderstood
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
PnB Rock
====Feelins
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
French Montana
====No Stylist
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
French Montana
====Unforgettable
	AZ failed
	ADDED LYRICS
French Montana
====Creep On Me
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
French Montana
====Welcome to the Party  - from Deadpool 2
	AZ failed
	Lyric Wikia failed
	Genius failed
French Montana
====Tip Toe 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
French Montana
====First Time
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
French Montana
====Olha A Exploso - Remix
	AZ failed
	Lyric Wikia failed
	Genius failed
French Montana
====Lockjaw
	AZ failed
	ADDED LYRICS
French Montana
====Welcome to the Party  - Valentino Khan Remi

====In My Life - Remastered 2009
	AZ failed
	Lyric Wikia failed
	Genius failed
The Beatles
====While My Guitar Gently Weeps - Remastered
	AZ failed
	Lyric Wikia failed
	Genius failed
Bastille
====Happier
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Bastille
====Quarter Past Midnight
	AZ failed
	ADDED LYRICS
Bastille
====Pompeii
	AZ failed
	ADDED LYRICS
Bastille
====World Gone Mad
	AZ failed
	ADDED LYRICS
Bastille
====Happier - Breathe Carolina Remix
	AZ failed
	Lyric Wikia failed
	Genius failed
Bastille
====Good Grief
	AZ failed
	ADDED LYRICS
Bastille
====Of The Night
	AZ failed
	ADDED LYRICS
Bastille
====Happier - Stripped
	AZ failed
	Lyric Wikia failed
	Genius failed
Bastille
====Oblivion
	AZ failed
	ADDED LYRICS
Bastille
====Send Them Off!
	AZ failed
	ADDED LYRICS
Led Zeppelin
====Stairway To Heaven
	AZ failed
	ADDED LYRICS
Led Zeppelin
====Immigrant Song
	AZ failed
	ADDED LYRICS
Led Zeppelin
====Whole Lotta Love
	AZ failed
	ADDED LYRICS
Led Zeppelin
====Black Dog
	AZ failed
	ADDED 

...

	ADDED LYRICS
Tyler Farr
====Whiskey in My Water
	AZ failed
	ADDED LYRICS
Tyler Farr
====Love by the Moon
	AZ failed
	Lyric Wikia failed
	Genius failed
Tyler Farr
====Withdrawals
	AZ failed
	ADDED LYRICS
Tyler Farr
====C.O.U.N.T.R.Y.
	AZ failed
	ADDED LYRICS
Randy Houser
====Runnin' Outta Moonlight
	AZ failed
	ADDED LYRICS
Randy Houser
====How Country Feels
	AZ failed
	ADDED LYRICS
Randy Houser
====We Went
	AZ failed
	ADDED LYRICS
Randy Houser
====Boots On
	AZ failed
	ADDED LYRICS
Randy Houser
====Like a Cowboy
	AZ failed
	ADDED LYRICS
Randy Houser
====No Stone Unturned
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Randy Houser
====Goodnight Kiss
	AZ failed
	ADDED LYRICS
Randy Houser
====What Whiskey Does 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Randy Houser
====New Buzz
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Randy Houser
====Mamma Don't Know
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Dylan Scott
====Hooked
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Dylan Scott
====My Gir

	Lyric Wikia failed
	ADDED LYRICS
John Legend
====Written In The Stars
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
John Legend
====Ordinary People
	AZ failed
	ADDED LYRICS
John Legend
====Beauty and the Beast
	AZ failed
	ADDED LYRICS
John Legend
====Love Me Now
	AZ failed
	ADDED LYRICS
John Legend
====You & I 
	AZ failed
	Lyric Wikia failed
	Genius failed
John Legend
====Save Room
	AZ failed
	ADDED LYRICS
John Legend
====Tonight 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
John Legend
====Start A Fire
	AZ failed
	ADDED LYRICS
Amy Winehouse
====Back To Black
	AZ failed
	ADDED LYRICS
Amy Winehouse
====Rehab
	AZ failed
	ADDED LYRICS
Amy Winehouse
====You Know I'm No Good
	AZ failed
	ADDED LYRICS
Amy Winehouse
====Tears Dry On Their Own
	AZ failed
	ADDED LYRICS
Amy Winehouse
====Valerie - Live At BBC Radio 1 Live Lounge, London / 2007
	AZ failed
	Lyric Wikia failed
		No results for: Valerie - Live At BBC Radio 1 Live Lounge, London / 2007
	Genius failed
Amy Winehouse
====Love Is A Losing

Anna Of The North
====Baby
	AZ failed
	ADDED LYRICS
Anna Of The North
====Sway
	AZ failed
	ADDED LYRICS
Anna Of The North
====Us - Acoustic
	AZ failed
	Lyric Wikia failed
	Genius failed
24hrs
====Lie Detector 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====Lowkey 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====Don't Mess 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====What You Like 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====Don't Sleep On Me 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====Fifty Ball 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====What You Like 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====Last Man
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====Met Gala 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
24hrs
====Leave LA 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
THEY.
====What I Know Now 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
THEY.
====Broken 
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
THEY.

	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Jamie Foxx
====Blame It
	AZ failed
	ADDED LYRICS
Jamie Foxx
====Slow Jamz
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Jamie Foxx
====Fall For Your Type
	AZ failed
	ADDED LYRICS
Jamie Foxx
====Unpredictable - Main
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Jamie Foxx
====You Changed Me
	AZ failed
	ADDED LYRICS
Jamie Foxx
====Winner
	AZ failed
	ADDED LYRICS
Jamie Foxx
====Just Like Me
	AZ failed
	ADDED LYRICS
Jamie Foxx
====The City's Yours
	AZ failed
	Lyric Wikia failed
	ADDED LYRICS
Jamie Foxx
====DJ Play A Love Song
	AZ failed
	ADDED LYRICS
Jamie Foxx
====Who Am I?
	AZ failed
	ADDED LYRICS
Macy Gray
====I Try
	AZ failed
	ADDED LYRICS
Macy Gray
====Over You
	AZ failed
	ADDED LYRICS
Macy Gray
====First Time
	AZ failed
	ADDED LYRICS
Macy Gray
====Sweet Baby 
	AZ failed
	ADDED LYRICS
Macy Gray
====Sugar Daddy
	AZ failed
	ADDED LYRICS
Macy Gray
====Still
	AZ failed
	ADDED LYRICS
Macy Gray
====Buddha
	AZ failed
	ADDED LYRICS
Macy Gray
====Whit

TypeError: cannot concatenate 'str' and 'float' objects